In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import GradientBoostingRegressor
import pickle
import csv
import warnings

warnings.filterwarnings("ignore")

### Auxiliary functions:

In [2]:
def load_dataset():
    '''
    Load train, valid and test dataset in a specified year.

    Returns:
    -----------------------------------
    (train_data, valid_data, test_data): a tuple of train, valid, test dataset. Each dataset is represented by a numpy array where each row is a trip sample (src, dst, count).
    '''
    # load train data
    train = pd.read_csv('data/LODES/CommutingFlow_2015_train_Denver.csv')
    # load valid data
    valid = pd.read_csv('data/LODES/CommutingFlow_2015_valid_Denver.csv')
    # load test data
    test = pd.read_csv('data/LODES/CommutingFlow_2015_test_Denver.csv')
    # mapping BoroCT2010 to node id
    mapping_table = pd.read_csv('data/CensusTract/mapping_NodeIDDenverCT.csv')
    train = ct2nid(train, mapping_table)
    valid = ct2nid(valid, mapping_table)
    test = ct2nid(test, mapping_table)
    # construct in/out flow count for training
    inflow_train = pd.DataFrame(index=mapping_table['node_id']) # container
    outflow_train = pd.DataFrame(index=mapping_table['node_id']) # container
    inflow = train.groupby('dst').agg({'count': 'sum'}) # stats
    outflow = train.groupby('src').agg({'count': 'sum'}) # stats
    inflow.index.name = 'node_id'
    outflow.index.name = 'node_id'
    inflow_train['count'] = inflow # pass the value
    outflow_train['count'] = outflow # pass the value
    inflow_train = inflow_train.fillna(0).sort_index() # fillna
    outflow_train = outflow_train.fillna(0).sort_index() # fillna
    # load node feature table
    node_feats = pd.read_csv('data/PLUTO/denver_ct_ratios2015.csv')
    node_feats['CT'] = mapping_table.set_index('CT').loc[node_feats['CT']].values # map census tract to node id
    node_feats = node_feats.rename(columns={'CT': 'nid'}).set_index('nid').sort_index()
    # sanity check
    if node_feats.isnull().values.any(): # if there is any NaN in nodes' feature table
        node_feats.fillna(0, inplace=True)
        warnings.warn('Feature table contains NaN. 0 is used to fill these NaNs')
    # normalization
    node_feats = (node_feats - node_feats.mean()) / node_feats.std()
    # load adjacency matrix
    ct_adj = pd.read_csv('data/CensusTract/adjacency_matrix_withweightDenver.csv', index_col=0)
    ct_inorder = mapping_table.sort_values(by='node_id')['CT']
    ct_adj = ct_adj.loc[ct_inorder, ct_inorder.astype(str)]
    # min-max scale the weights
    ct_adj = ct_adj / ct_adj.max().max() # min is 0
    # fill nan with 0
    ct_adj = ct_adj.fillna(0)
    # load distance matrix
    distm = pd.read_csv('data/OSRM/census_tract_trip_duration_matrix_Denver.csv', index_col='CT')
    # define column order
    cols = ['src', 'dst', 'count']
    # return
    data = {}
    data['train'] = train[cols].values
    data['valid'] = valid[cols].values
    data['test'] = test[cols].values
    data['train_inflow'] = inflow_train.values
    data['train_outflow'] = outflow_train.values
    data['num_nodes'] = ct_adj.shape[0]
    data['node_feats'] = node_feats.values
    data['ct_adjacency_withweight'] = ct_adj.values
    data['distm'] = distm.values
    return data

In [3]:
def ct2nid(dataframe, mapping_table):
    '''
    Mapping BoroCT2010 code to node id.

    Inputs: 
    -----------------------------------
    dataframe: The dataframe of trips. it is supposed to have 3 columns: h_BoroCT2010, w_BoroCT2010, count
    mapping_table: The table of mapping between node id and BoroCT2010. The table is supposed to have two columns: node_id, BoroCT2010

    Returns:
    -----------------------------------
    frame: it is supposed to have 3 columns: src, dst, count. src and dst are the node id of source and target respectively.
    '''
    frame = dataframe.copy()
    mapping = mapping_table.copy()
    # do the mapping
    mapping = mapping.set_index('CT')
    frame['src'] = mapping.loc[frame['h_geocode']].values
    frame['dst'] = mapping.loc[frame['w_geocode']].values
    # return
    return frame[['src', 'dst', 'count']]

In [4]:
def construct_feat_from_srcemb_dstemb_dist(triplets, src_emb, dst_emb, dist):
    feat_src = src_emb[triplets[:, 0]]
    feat_dst = dst_emb[triplets[:, 1]]
    feat_dist = dist[triplets[:, 0], triplets[:, 1]].reshape(-1, 1)
    X = np.concatenate([feat_src, feat_dst, feat_dist], axis=1)
    y = triplets[:, 2]
    return X, y

In [5]:
def RMSE(y_hat, y):
    '''
    Root Mean Square Error Metric
    '''
    return np.sqrt(np.mean((y_hat - y)**2))

def CPC(y_hat, y):
    '''
    Common Part of Commuters Metric
    '''
    common = np.min(np.stack((y_hat, y), axis=1), axis=1)
    return 2 * np.sum(common) / (np.sum(y_hat) + np.sum(y))

def CPL(y_hat, y):
    '''
    Common Part of Links Metric. 
    
    Check the topology.
    '''
    yy_hat = y_hat > 0
    yy = y > 0
    return 2 * np.sum(yy_hat * yy) / (np.sum(yy_hat) + np.sum(yy))

def MAPE(y_hat, y):
    '''
    Mean Absolute Percentage Error Metric
    '''
    abserror = np.abs(y_hat - y)
    return np.mean(abserror / y)

def MAE(y_hat, y):
    '''
    Mean Absolute Error Metric
    '''
    abserror = np.abs(y_hat - y)
    return np.mean(abserror)

def NRMSE(RMSE, y):
    '''
    Normalized Root Mean Square Error Metric
    '''
    return RMSE/(y.max() - y.min())

def evaluate(y_hat, y):
    '''
    Evaluate the error in different metrics
    '''
    # metric
    rmse = RMSE(y_hat, y)
    mae = MAE(y_hat, y)
    mape = MAPE(y_hat, y)
    cpc = CPC(y_hat, y)
    cpl = CPL(y_hat, y)
    nrmse = NRMSE(rmse,y)
    # return
    return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'CPC': cpc, 'CPL': cpl, 'NRMSE':nrmse}

## Experiment

In [29]:
metric = "mahalanobis"

# "manhattan"
# "euclidean"
# "chebyshev"
# "minkowski"
# "wminkowski"
# "seuclidean"
# "mahalanobis"

In [30]:
# Reading files
cityA_features = pd.read_csv('data/PLUTO/columbus_ct_ratios2015.csv')
cityB_features = pd.read_csv('data/PLUTO/denver_ct_ratios2015.csv')

cityA_features = cityA_features[cityB_features.columns]

#### Find nearest neighbors:

In [32]:
# Find the nearest neighbors
X_train = cityA_features.drop(columns=['CT'])
y_train = cityA_features['CT']

knn = NearestNeighbors(metric=metric, metric_params={'V': np.cov(X_train, rowvar=False)})
knn.fit(X_train, y_train)

X_test = cityB_features.drop(columns=['CT'])
y_test = cityB_features['CT']

list_ct, list_neighbors = [], []
for ix in cityB_features.index:
  list_ct.append(y_test[ix])
  list_neighbors.append(knn.kneighbors(X_test.iloc[ix,:].values.reshape(1, -1))[1])

data = {'ct_cityB': list_ct, 'neighbors_cityA': list_neighbors}

df = pd.DataFrame(data)

#### Preprocess dataset:

In [33]:
# load dataset
data = load_dataset()
# parse dataset
train_data = data['train']
valid_data = data['valid']
test_data = data['test']
distm = data['distm']

#### Create new src and dst embeddings:

In [34]:
# load embeddings
epath = 'embeddings/censustract_embeddings_year2015_layers1_emb128_multitask(0.5, 0.25, 0.25).npz'
embeddings = np.load(epath, allow_pickle=True)
# parse embeddings
src_emb, dst_emb = embeddings['arr_0'], embeddings['arr_1']
# scale distance matrix
scaled_distm = distm / distm.max() * np.max([src_emb.max(), dst_emb.max()])
# construct src and dst embeddings to city B from city A
src_emb_new, dst_emb_new = [], []

for neigh in df['neighbors_cityA']:
    src_emb_new.append(src_emb[neigh[0]].mean(axis=0))
    dst_emb_new.append(dst_emb[neigh[0]].mean(axis=0))

src_emb_new = np.array(src_emb_new)
src_emb_new = np.array(src_emb_new)

# construct sample features
X_train, y_train = construct_feat_from_srcemb_dstemb_dist(train_data, src_emb_new, src_emb_new, scaled_distm)
X_valid, y_valid = construct_feat_from_srcemb_dstemb_dist(valid_data, src_emb_new, src_emb_new, scaled_distm)
X_test, y_test = construct_feat_from_srcemb_dstemb_dist(test_data, src_emb_new, src_emb_new, scaled_distm)

#### Training model:

In [35]:
# train gbrt
gbrt = GradientBoostingRegressor(max_depth=2, random_state=2019, n_estimators=100)
gbrt.fit(X_train, y_train)
# test
y_gbrt = gbrt.predict(X_test)
res = evaluate(y_gbrt, y_test)
# save result
with open('outputs/overall_performance.csv', 'a+') as fout:
    # create writer
    cols = ['metric', 'model','RMSE', 'MAE', 'MAPE', 'CPC', 'CPL','NRMSE']
    writer = csv.DictWriter(fout, fieldnames=cols)
    # create model name
    model_name = 'GMEL_year2019_layers1_emb128_multitask(0.5, 0.25, 0.25)'
    res['metric'] = metric
    res['model'] = model_name
    # write result
    writer.writerow(res)
# # save model
# with open('models/gbrt_year2015_layers1_emb128_multitask(0.5, 0.25, 0.25).txt', 'a+') as fout:
#     pickle.dump(gbrt, fout)

In [36]:
results = pd.read_csv('outputs/overall_performance.csv', names=['model','RMSE', 'MAE', 'MAPE', 'CPC', 'CPL','NRMSE'])
results.head()

,model,RMSE,MAE,MAPE,CPC,CPL,NRMSE
euclidean,"GMEL_year2019_layers1_emb128_multitask(0.5, 0....",8.648222,4.451674,1.305319,0.683602,0.999820,0.058831
manhattan,"GMEL_year2019_layers1_emb128_multitask(0.5, 0....",8.691705,4.462365,1.337679,0.683273,0.999820,0.059127
chebyshev,"GMEL_year2019_layers1_emb128_multitask(0.5, 0....",8.936493,4.550395,1.356366,0.676507,0.999461,0.060792
minkowski,"GMEL_year2019_layers1_emb128_multitask(0.5, 0....",8.648222,4.451674,1.305319,0.683602,0.999820,0.058831
mahalanobis,"GMEL_year2019_layers1_emb128_multitask(0.5, 0....",10.145984,5.445827,1.756627,0.613205,1.000000,0.069020
